## libraries


In [1]:
import os
import copy
import pandas as pd
from biopandas.pdb import PandasPdb
from Bio.Data import IUPACData
from Bio.Seq import Seq

%matplotlib inline
%config InlineBackend.figure_format='retina'

## inputs


In [2]:
ROOT = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
SUPPORT_DIR = os.path.join(ROOT,"support")
RESULTS_DIR = os.path.join(ROOT,'results','splicing_dependency_drugs')
pdb_inc_file = os.path.join(SUPPORT_DIR,"MDM2_HsaEX0038400-iso0-included_unrelaxed_rank_1_model_3.pdb")
pdb_exc_file = os.path.join(SUPPORT_DIR,"MDM2_HsaEX0038400-iso0-excluded_unrelaxed_rank_1_model_3.pdb")
metadata_file = os.path.join(RESULTS_DIR,'files','structure_inference','proteoforms_merged.tsv.gz')
gene_oi = "MDM2"
seq_id = "MDM2_HsaEX0038400-iso0"
margin = 10

## load

In [3]:
metadata = pd.read_table(metadata_file)
df_inc = PandasPdb().read_pdb(pdb_inc_file)
df_exc = PandasPdb().read_pdb(pdb_exc_file)
pdb_inc = open(pdb_inc_file,'r').read()
pdb_exc = open(pdb_exc_file,'r').read()

## preprocess

In [4]:
# make proper dataframes
## included
df_inc = df_inc.df["ATOM"][["residue_name","chain_id","residue_number"]].drop_duplicates()
df_inc["residue_title"] = df_inc["residue_name"].str.title()
df_inc["residue_symbol"] = df_inc["residue_title"].map(IUPACData.protein_letters_3to1)
seq_inc = ''.join(df_inc.loc[df_inc["chain_id"]=="A", "residue_symbol"].to_list())
res_inc = df_inc.loc[df_inc["chain_id"]=="A", "residue_number"].to_list()

## excluded
df_exc = df_exc.df["ATOM"][["residue_name","chain_id","residue_number"]].drop_duplicates()
df_exc["residue_title"] = df_exc["residue_name"].str.title()
df_exc["residue_symbol"] = df_exc["residue_title"].map(IUPACData.protein_letters_3to1)
seq_exc = ''.join(df_exc.loc[df_exc["chain_id"]=="A", "residue_symbol"].to_list())
res_exc = df_exc.loc[df_exc["chain_id"]=="A", "residue_number"].to_list()

print(seq_inc, df_inc.groupby(["chain_id"]).size(), df_inc.head())
print(seq_exc, df_exc.groupby(["chain_id"]).size(), df_exc.head())

MCNTNMSVPTDGAVTTSQIPASEQETLVRPKPLLLKLLKSVGAQKDTYTMKEVLFYLGQYIMTKRLYDEKQQHIVYCSNDLLGDLFGVPSFSVKEHRKIYTMIYRNLVVVNQQESSDSGTSVSENRCHLEGGSDQKDLVQELQEEKPSSSHLVSRPSTSSRRRAISETEENSDELSGERQRKRHKSDSISLSFDESLALCVIREICCERSSSSESTGTPSNPDLDAGVSEHSGDWLDQDSVSDQFSVEFEVESLDSEDYSLSEEGQELSDEDDEVYQVTVYQAGESDTDSFEEDPEISLADYWKCTSCNEMNPPLPSHCNRCWALRENWLPEDKGKDKGEISEKAKLENSTQAEEGFDVPDCKKTIVNDSRESCVEENDDKITQASQSQESEDYSQPSTSSSIIYSSQEDVKEFEREETQDKEESVESSLPLNAIEPCVICQGRPKNGCIVHGKTGHLMACFTCAKKLKKRNKPCPVCRQPIQMIVLTYFP chain_id
A    491
dtype: int64    residue_name chain_id  residue_number residue_title residue_symbol
0           MET        A               1           Met              M
8           CYS        A               2           Cys              C
14          ASN        A               3           Asn              N
22          THR        A               4           Thr              T
29          ASN        A               5           Asn              N
MCNTNMSVPTDGAVTTSQIPASEQETLVLFYLGQYIMTKRLYDEKQQHIVYCSNDLL

## get positions of aminoacids to highlight

In [5]:
# exon sequences
seqs_oi = metadata.loc[metadata["GENE"]==gene_oi, ["EVENT","event_aa"]].drop_duplicates().set_index("EVENT").to_dict()["event_aa"]

# get locations of events
## find
locs_oi_inc = {event: seq_inc.find(aa) for event, aa in seqs_oi.items()}
## remove not found
locs_oi_inc = {event: loc_start for event, loc_start in locs_oi_inc.items() if loc_start!=(-1)}
## add end
locs_oi_inc = {event: {"start": loc_start, "end": loc_start+len(seqs_oi[event])} for event, loc_start in locs_oi_inc.items()}
## add margins
locs_oi_inc = {
    event: {
        "start": loc["start"], 
        "end": loc["end"],
        "upstream_start": loc["start"] - margin - 1,
        "upstream_end": loc["start"] - 1,
        "downstream_start": loc["end"] + 1,
        "downstream_end": loc["end"] + margin + 1
    } 
    for event, loc in locs_oi_inc.items()
}

## locations of excluded isoform
locs_oi_exc = copy.deepcopy(locs_oi_inc)
for event in locs_oi_exc.keys():
    locs_oi_exc[event]["downstream_start"] = locs_oi_exc[event]["downstream_start"] - len(seqs_oi[event]) - 1
    locs_oi_exc[event]["downstream_end"] = locs_oi_exc[event]["downstream_end"] - len(seqs_oi[event]) - 1

print(seqs_oi)
print(locs_oi_inc)
print(locs_oi_exc)

{'HsaEX0038400': 'VRPKPLLLKLLKSVGAQKDTYTMKE', 'HsaEX0038402': 'RWSFTMLPRLVWNSWAQGICLPRPPKVLDLQ'}
{'HsaEX0038400': {'start': 27, 'end': 52, 'upstream_start': 16, 'upstream_end': 26, 'downstream_start': 53, 'downstream_end': 63}}
{'HsaEX0038400': {'start': 27, 'end': 52, 'upstream_start': 16, 'upstream_end': 26, 'downstream_start': 27, 'downstream_end': 37}}


## prepare color palettes

In [6]:
colors_inc = {}
colors_exc = {}
for event in locs_oi_exc.keys():
    # included isoform
    colors_inc[event] = {}
    for idx, res in enumerate(res_inc):
        # check upstream
        if (idx >= locs_oi_inc[event]["upstream_start"]) & (idx <= locs_oi_inc[event]["upstream_end"]):
            colors_inc[event][res] = "green"
        # check in exon
        elif (idx >= locs_oi_inc[event]["start"]) & (idx <= locs_oi_inc[event]["end"]):
            colors_inc[event][res] = "orange"
        # check downstream
        elif (idx >= locs_oi_inc[event]["downstream_start"]) & (idx <= locs_oi_inc[event]["downstream_end"]):
            colors_inc[event][res] = "blue"
        else:
            colors_inc[event][res] = "gray"

    # excluded isoform
    colors_exc[event] = {}
    for idx, res in enumerate(res_exc):
        # check upstream
        if (idx >= locs_oi_exc[event]["upstream_start"]) & (idx <= locs_oi_exc[event]["upstream_end"]):
            colors_exc[event][res] = "green"
        # check downstream
        elif (idx >= locs_oi_exc[event]["downstream_start"]) & (idx <= locs_oi_exc[event]["downstream_end"]):
            colors_exc[event][res] = "blue"
        else:
            colors_exc[event][res] = "gray"
            
print(colors_inc)
print(colors_exc)

{'HsaEX0038400': {1: 'gray', 2: 'gray', 3: 'gray', 4: 'gray', 5: 'gray', 6: 'gray', 7: 'gray', 8: 'gray', 9: 'gray', 10: 'gray', 11: 'gray', 12: 'gray', 13: 'gray', 14: 'gray', 15: 'gray', 16: 'gray', 17: 'green', 18: 'green', 19: 'green', 20: 'green', 21: 'green', 22: 'green', 23: 'green', 24: 'green', 25: 'green', 26: 'green', 27: 'green', 28: 'orange', 29: 'orange', 30: 'orange', 31: 'orange', 32: 'orange', 33: 'orange', 34: 'orange', 35: 'orange', 36: 'orange', 37: 'orange', 38: 'orange', 39: 'orange', 40: 'orange', 41: 'orange', 42: 'orange', 43: 'orange', 44: 'orange', 45: 'orange', 46: 'orange', 47: 'orange', 48: 'orange', 49: 'orange', 50: 'orange', 51: 'orange', 52: 'orange', 53: 'orange', 54: 'blue', 55: 'blue', 56: 'blue', 57: 'blue', 58: 'blue', 59: 'blue', 60: 'blue', 61: 'blue', 62: 'blue', 63: 'blue', 64: 'blue', 65: 'gray', 66: 'gray', 67: 'gray', 68: 'gray', 69: 'gray', 70: 'gray', 71: 'gray', 72: 'gray', 73: 'gray', 74: 'gray', 75: 'gray', 76: 'gray', 77: 'gray', 78: 

## visualize

In [32]:
import py3Dmol

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js', width=400, height=400)

In [33]:
view.removeAllModels()
view.addModel(pdb_inc_file,'pdb')

view.setBackgroundColor('black', 1.0)
view.setStyle({'chain':'A'}, {'cartoon':{'colorscheme':{'prop':'resi','map':colors_inc["HsaEX0038400"]}}})

view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [31]:
view.removeAllModels()
view.addModel(pdb_exc_file,'pdb')

view.setBackgroundColor('black', 1.0)
view.setStyle({'chain':'A'}, {'cartoon':{'colorscheme':{'prop':'resi','map':colors_exc["HsaEX0038400"]}}})

view.zoomTo()
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol